# 8. Introduction to Demand-Side Agents and Flexibility in the ASSUME Framework

In this tutorial, we will explore the concept of **Demand-Side Agents (DSA)** and their role in managing electricity consumption in response to price signals. The focus will be on modeling flexibility and performing **Power Flow Analysis** after clearing the day-ahead market. 

In modern electricity markets, **Demand-Side Management (DSM)** plays an essential role in ensuring efficient energy consumption, maintaining grid stability, and optimizing costs. Demand-side agents, such as industrial plants or residential units, can adjust their energy consumption in response to price signals from the electricity market. This capability to adapt energy usage in real-time provides **flexibility** that is crucial for balancing supply and demand.

In the **ASSUME Framework**, demand-side agents are modeled as entities with specific energy requirements, flexibility capabilities, and interaction mechanisms in the market. These agents are able to:
- **Participate in the electricity market** by submitting bids to purchase electricity based on their demand.
- **Respond to price signals** to adjust their electricity consumption (flexibility).
- **Perform load-shifting** by changing their consumption behavior in response to electricity prices, optimizing their costs while supporting grid operations.

### Key Concepts:
1. **Demand-Side Agents**: Entities that consume electricity, like industrial plants or households. These agents can adjust their consumption based on real-time market prices.
2. **Flexibility**: The capability of an agent to adapt its electricity consumption, either by increasing or decreasing its load, in response to market conditions or price signals.
3. **Power Flow Analysis**: After the day-ahead market clears, power flow analysis is conducted to ensure that electricity is distributed across the network in a manner that respects grid constraints and prevents congestion.

### Integration of the Characteristics of the Agent:
- Each demand-side agent is modeled with **specific attributes** such as **rated power**, **minimum operating power**, and **flexibility measures**. These characteristics dictate how the agent behaves in the electricity market.
- Agents can also be equipped with technologies like **electrolyser**, **DRI plant**, and **electric arc furnace**, each contributing to the agent’s overall power demand profile.

### In this tutorial, we will:
1. **Introduce how demand-side agents are modeled in the ASSUME framework**:
   - We'll model a **steel plant** as a demand-side agent, including the specific characteristics such as rated power, minimum power, and flexibility settings.
2. **Explore how flexibility is implemented**, allowing agents to shift their energy usage:
   - We'll show how the steel plant can dynamically respond to market prices by adjusting its electricity consumption using predefined bidding strategies.
3. **Model a steel plant as a demand-side agent** and simulate its participation in the electricity market:
   - The steel plant will participate in the day-ahead market, submitting bids and optimizing its electricity consumption.
4. **Integrate the characteristics and inputs of the steel plant agent**
5. **Integrate in ASSUME**:
   - We'll configure the **ASSUME framework**, setting up the market and integrating the steel plant agent for simulation. This will involve defining the market configuration, demand, supply, and load-shifting behaviors in response to price signals.
6. **Analyze the results**:
   - After running the simulation, we'll review how the steel plant performed in the electricity market. This will include analyzing energy consumption, bidding results, and flexibility behaviors.
7. **Analyze grid congestion**:
   - We'll perform a power flow analysis to evaluate how energy flows through the grid after the day-ahead market clears. We'll check for any grid congestion and determine how the demand-side agent’s behavior impacts the overall grid.

## 2. Setting Up the ASSUME Framework

Before diving into **DSA** and **Flexibility**, ensure that you have the ASSUME framework installed and set up correctly. If you haven't done so already, follow the steps below to install the ASSUME core package and clone the repository containing predefined scenarios.

**Note:** If you already have the ASSUME framework installed and the repository cloned, you can skip executing the following code cells.

In [1]:
# Install the ASSUME framework
# !pip install assume-framework

# Install Plotly if not already installed
# !pip install plotly

# Clone the ASSUME repository containing predefined scenarios
# !git clone https://github.com/assume-framework/assume.git assume-repo

Let's also import some basic libraries that we will use throughout the tutorial.

In [11]:
# import plotly for visualization
import pyomo as pyo

# import yaml for reading and writing YAML files
# Function to display DataFrame in Jupyter
from assume.units.dst_components import (
    create_driplant,
    create_dristorage,
    create_electric_arc_furnance,
    create_electrolyser,
    create_hydrogen_storage,
)

# Import components and units for the steel plant

## 3. Defining the Steel Plant as a Demand-Side Agent

The Steel Plant is an example of an industrial demand-side agent in the ASSUME framework. This agent has specific characteristics such as rated power, flexibility, and the ability to adjust its electricity consumption based on price signals. We will now explore how to define the Steel Plant in the framework.

**Steel Plant: Overview and Characteristics**

The Steel Plant is an industrial agent that consists of several key components (e.g., DRI plant, electrolyser, EAF, etc.). These components consume electricity and can be modeled to react dynamically to market conditions.

In the ASSUME framework, the steel plant agent is created by defining its characteristics, components, and objectives. Let's start by defining the Steel Plant with its core characteristics:

In [20]:
# class SteelPlant(SupportsMinMax, DSMFlex):
#     """
#     Class to define the Steel Plant as a demand-side agent in the ASSUME framework.
#     """

#     def __init__(
#         self,
#         id: str,
#         unit_operator: str,
#         bidding_strategies: dict,
#         technology: str = "steel_plant",
#         node: str = "node0",
#         index: pd.DatetimeIndex = None,
#         location: tuple[float, float] = (0.0, 0.0),
#         components: dict[str, dict] = None,
#         objective: str = None,
#         flexibility_measure: str = "max_load_shift",
#         demand: float = 0,
#         cost_tolerance: float = 10,
#         **kwargs,
#     ):
#         super().__init__(
#             id=id,
#             unit_operator=unit_operator,
#             technology=technology,
#             bidding_strategies=bidding_strategies,
#             index=index,
#             node=node,
#             location=location,
#             **kwargs,
#         )

Defining Characteristics:

    id: The unique identifier for the steel plant agent.
    unit_operator: The entity operating the steel plant.
    bidding_strategies: Defines the market bidding behavior (such as NaiveDASteelplantStrategy).
    technology: Represents the type of technology used, here set to "steel_plant".
    node: Specifies the grid connection point for the steel plant.
    flexibility_measure: Indicates the flexibility strategy, such as load-shifting capabilities.
    demand: The energy demand of the steel plant.
    cost_tolerance: The tolerance level for cost variations.

### Bringing `dst_components` into the Steel Plant Process

In the steel plant, we use components like the **electrolyser** and **hydrogen storage** to model the production and storage of hydrogen, which plays a critical role in decarbonized steel production. These components are imported from `dst_components.py` and integrated into the plant's process.

In this section, we will showcase how to model these components, define their characteristics, and integrate them into the overall process of the steel plant.

#### 1. **Electrolyser**

The **electrolyser** produces hydrogen through electrolysis, using electricity as the input. The electrolyser's constraints ensure that it operates within its rated power capacity, follows ramp rates for power changes, and has operational limits like minimum operating time and efficiency.

We will use the `create_electrolyser` function from `dst_components.py` to add the electrolyser to the steel plant's Pyomo model.

#### 2. **Hydrogen Storage**

The **hydrogen storage** component is used to store hydrogen produced by the electrolyser. It helps manage the supply and ensures that hydrogen is available when needed. The hydrogen storage has parameters like **maximum capacity**, **storage loss rate**, and **charge/discharge rates**.

We will use the `create_hydrogen_storage` function from `dst_components.py` to integrate hydrogen storage into the model.


### Modeling Components in the ASSUME Framework

In the **ASSUME** framework, components like the **electrolyser** and **hydrogen storage** are modeled using Pyomo, a Python-based optimization modeling tool. The framework supports detailed modeling of each component by specifying their characteristics and operational constraints.

For each component, attributes such as **rated power**, **ramp rates**, and **efficiency** are defined. These attributes are essential for simulating the component's behavior in the energy system.

#### Example: Electrolyser Model
The **electrolyser** is a crucial component in hydrogen production. In this framework, the electrolyser is modeled with various characteristics, including power limits, operational efficiency, and ramp rates. These attributes ensure that the electrolyser operates within its technical and economic boundaries.

Here's how we define the electrolyser's model:


In [24]:
# def create_electrolyser(
#     model,
#     rated_power,
#     min_power,
#     ramp_up,
#     ramp_down,
#     min_operating_time,
#     min_down_time,
#     efficiency,
#     time_steps,
#     **kwargs,
# ):
#     """
#     Creates the electrolyser component in the Pyomo model with defined characteristics.
#     """
#     # Define a Pyomo Block for the electrolyser component
#     model_part = pyo.Block()

#     # Define the attributes of the electrolyser component
#     model_part.rated_power = pyo.Param(initialize=rated_power)
#     model_part.min_power = pyo.Param(initialize=min_power)
#     model_part.ramp_up = pyo.Param(initialize=ramp_up)
#     model_part.ramp_down = pyo.Param(initialize=ramp_down)
#     model_part.efficiency = pyo.Param(initialize=efficiency)

#     # Define the Pyomo variables for operational behavior
#     model_part.power = pyo.Var(time_steps, bounds=(0, rated_power))
#     model_part.hydrogen_out = pyo.Var(time_steps, domain=pyo.NonNegativeReals)

#     # Define the Pyomo constraints
#     @model.Constraint(time_steps)
#     def min_operating_time_constraint(m, t):
#         """
#         Ensures that the electrolyser operates at or above its minimum power.
#         """
#         return model_part.power[t] >= model_part.min_power

#     @model.Constraint(time_steps)
#     def ramp_rate_constraints(m, t):
#         """
#         Ensures that the electrolyser respects the ramp up and ramp down constraints.
#         """
#         if t > 0:
#             return (
#                 model_part.power[t] - model_part.power[t - 1] <= model_part.ramp_up,
#                 model_part.power[t - 1] - model_part.power[t] <= model_part.ramp_down,
#             )

#     # Define hydrogen output based on the efficiency
#     @model.Constraint(time_steps)
#     def hydrogen_production_constraint(m, t):
#         """
#         Ensures that hydrogen output is proportional to the power input based on efficiency.
#         """
#         return model_part.hydrogen_out[t] == model_part.power[t] * model_part.efficiency

### 4. Initializing the Components and Process Sequence of the Steel Plant

The steel plant consists of multiple components, such as the **DRI plant**, **electrolyser**, **Electric Arc Furnace (EAF)**, and **hydrogen storage**. These components are initialized and connected into a process sequence to define how the steel plant operates.

#### Mapping of Component Type Identifiers to Their Respective Classes

Before initializing the components and process sequences, the steel plant needs to map the component types to their respective functions. This is done using the following dictionary, which links the component type identifiers (e.g., `"electrolyser"`, `"eaf"`) to the respective creation functions in the `dst_components.py` file:


In [17]:
dst_components = {
    "electrolyser": create_electrolyser,
    "h2storage": create_hydrogen_storage,
    "dri_plant": create_driplant,
    "dri_storage": create_dristorage,
    "eaf": create_electric_arc_furnance,
}


- The `def initialize_components(components)` function calls the Pyomo models of the respective components.
- The `def initialize_process_sequence()` function handles the connection of the entire process, ensuring that the plant operates in a defined sequence.

To initialize the components of the steel plant, we use the `initialize_components()` function. This function iterates through the list of components, calls the corresponding factory method for each component (as mapped in `dst_components`), and transfers the necessary attributes to the Pyomo model.


In [21]:
# Initialize components of the steel plant
def initialize_components(self, components: dict[str, dict]):
    """
    Initializes the components of the steel plant.

    Args:
        components (dict[str, dict]): The components of the steel plant.
        model (pyomo.ConcreteModel): The Pyomo model.
    """
    self.model.dsm_blocks = pyo.Block(list(components.keys()))
    for technology, component_data in components.items():
        if technology in dst_components:
            factory_method = dst_components[technology]
            self.model.dsm_blocks[technology].transfer_attributes_from(
                factory_method(
                    self.model, time_steps=self.model.time_steps, **component_data
                )
            )

In this function:

- The `components` argument is a dictionary where the keys are the component types (e.g., `"electrolyser"`, `"eaf"`) and the values are dictionaries of component-specific parameters.
- For each component, the factory method corresponding to the component type is called (e.g., `create_electrolyser` for an electrolyser).
- The attributes of the initialized component are then transferred to the Pyomo model using the `transfer_attributes_from()` function.


### Initializing the Process Sequence for the Steel Plant

The `initialize_process_sequence()` function is responsible for defining how the different components of the steel plant are connected to form a complete process. This function ensures that the flow of materials (such as hydrogen and direct reduced iron (DRI)) between components, such as the **electrolyser**, **hydrogen storage**, **DRI plant**, and **Electric Arc Furnace (EAF)**, is properly constrained.

In [22]:
# Initialize components of the steel plant
def initialize_process_sequence(self):
    """
    Initializes the process sequence and constraints for the steel plant. Here, the components/ technologies are connected to establish a process for steel production
    """
    # Assuming the presence of 'h2storage' indicates the desire for dynamic flow management
    has_h2storage = "h2storage" in self.model.dsm_blocks.keys()

    # Constraint for direct hydrogen flow from Electrolyser to dri plant
    @self.model.Constraint(self.model.time_steps)
    def direct_hydrogen_flow_constraint(m, t):
        """
        Ensures the direct hydrogen flow from the electrolyser to the DRI plant or storage.
        """
        # This constraint allows part of the hydrogen produced by the dri plant to go directly to the EAF
        # The actual amount should ensure that it does not exceed the capacity or demand of the EAF
        if has_h2storage:
            return (
                self.model.dsm_blocks["electrolyser"].hydrogen_out[t]
                + self.model.dsm_blocks["h2storage"].discharge[t]
                == self.model.dsm_blocks["dri_plant"].hydrogen_in[t]
                + self.model.dsm_blocks["h2storage"].charge[t]
            )
        else:
            return (
                self.model.dsm_blocks["electrolyser"].hydrogen_out[t]
                >= self.model.dsm_blocks["dri_plant"].hydrogen_in[t]
            )

    # Assuming the presence of dristorage' indicates the desire for dynamic flow management
    has_dristorage = "dri_storage" in self.model.dsm_blocks.keys()

    # Constraint for direct hydrogen flow from Electrolyser to dri plant
    @self.model.Constraint(self.model.time_steps)
    def direct_dri_flow_constraint(m, t):
        """
        Ensures the direct DRI flow from the DRI plant to the EAF or DRI storage.
        """
        # This constraint allows part of the dri produced by the dri plant to go directly to the dri storage
        # The actual amount should ensure that it does not exceed the capacity or demand of the EAF
        if has_dristorage:
            return (
                self.model.dsm_blocks["dri_plant"].dri_output[t]
                + self.model.dsm_blocks["dri_storage"].discharge_dri[t]
                == self.model.dsm_blocks["eaf"].dri_input[t]
                + self.model.dsm_blocks["dri_storage"].charge_dri[t]
            )
        else:
            return (
                self.model.dsm_blocks["dri_plant"].dri_output[t]
                == self.model.dsm_blocks["eaf"].dri_input[t]
            )

    # Constraint for material flow from dri plant to Electric Arc Furnace
    @self.model.Constraint(self.model.time_steps)
    def shaft_to_arc_furnace_material_flow_constraint(m, t):
        """
        Ensures the material flow from the DRI plant to the Electric Arc Furnace.
        """
        return (
            self.model.dsm_blocks["dri_plant"].dri_output[t]
            == self.model.dsm_blocks["eaf"].dri_input[t]
        )

### Key Process Sequence Steps:

- **Hydrogen Flow**:
    - The `direct_hydrogen_flow_constraint()` ensures that hydrogen produced by the electrolyser flows to the DRI plant or hydrogen storage.
    - If hydrogen storage is present, it can charge or discharge hydrogen, ensuring that the electrolyser's output is properly distributed.

- **DRI Flow**:
    - The `direct_dri_flow_constraint()` manages the flow of direct reduced iron (DRI) from the DRI plant to the EAF or DRI storage.
    - If DRI storage is present, it can charge or discharge DRI, ensuring that the DRI output from the plant is managed efficiently.

- **Material Flow to EAF**:
    - The `shaft_to_arc_furnace_material_flow_constraint()` ensures that the material produced by the DRI plant flows directly into the Electric Arc Furnace (EAF) for further steel production.

These constraints ensure the proper flow of materials between the steel plant’s components, supporting efficient steel production while maintaining the operational constraints of each technology.


### 6. Executing the Optimization

To execute the optimization process, the function `determine_optimal_operation_without_flex()` is called. This function computes the optimal operation for the steel plant under the defined objectives.


In [23]:
def determine_optimal_operation_without_flex(self):
    """
    Execute the optimization for the steel plant based on the current market conditions.
    This function will simulate the plant's operation without considering flexibility.
    """